Name : Hosam ksbaa
Id 221002006 1

Project 1

-   The `spark.conf` object is used to get and set Spark configuration parameters.
-   The `spark.sql.shuffle.partitions` parameter is retrieved using the `spark.conf.get()` function.
-   The `spark.sql.shuffle.partitions` parameter is set to 8 using the `spark.conf.set()` function.
-   The `spark.sql.files.minPartitionlun` parameter is also set to 8 using the `spark.conf.set()` function.
-   The values of these parameters are retrieved again using the `spark.conf.get()` function.

In [ ]:


from pyspark.sql. functions import *

spark.conf.get("spark.sql.shuffle.partitions")

spark.conf.set("spark.sql.shuffle.partitions","8")
spark.conf.set("spark.sql.files.minPartitionlun","8")

spark.conf.get("spark.sql.shuffle.partitions")





Out[42]: '8'

1 - Laoading Data

In [ ]:
file_location = "/FileStore/tables/sample.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)
# display(df)

In [ ]:
df.printSchema()


root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_time_in_secs: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)



2 - Cleaning the data
-   The first line drops four columns, namely "medallion," "vendor_id," "rate_code," and "store_and_fwd_flag," from the DataFrame called "df."
-   The second line creates two new columns in the DataFrame, "pickup_timeStamp" and "dropoff_timeStamp," using the "to_timestamp" function on the "pickup_datetime" and "dropoff_datetime" columns, respectively. This function converts the strings in those columns to timestamps, which are more usable for various data analysis tasks.
-   The third line drops the original "dropoff_datetime" and "pickup_datetime" columns, which are no longer needed in their original string format.

So overall, this code is cleaning and manipulating a DataFrame by dropping unnecessary columns and creating new timestamp columns from existing columns.

In [ ]:
df=df.drop("medallion","vendor_id","rate_code","store_and_fwd_flag")
df = df.withColumn("pickup_timeStamp",to_timestamp("pickup_datetime"))\
         .withColumn("dropoff_timeStamp",to_timestamp("dropoff_datetime"))
df =df.drop("dropoff_datetime","pickup_datetime")


# display(df4)

3 - setup the 2nd file

In [ ]:
!pip install urlopen
!pip install shapely


You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-6fe89238-3942-48c5-a4e3-b83624c85ea0/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 2.3 MB 5.8 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-6fe89238-3942-48c5-a4e3-b83624c85ea0/bin/python -m pip install --upgrade pip' command.


In [ ]:
import urllib, json
from urllib.request import urlopen

from shapely.geometry import shape, Point 
from pyspark.sql.functions import udf 
from pyspark.sql.types import StringType 


json_url = urlopen("https://raw.githubusercontent.com/HosamKsbaa/z/main/nyc-boroughs.geojson")
data = json.loads(json_url.read()) 
# The code is attempting to load a geojson file of New York City boroughs from a URL using the "urllib" library.
# becouse , for some reason, the file could not be loaded from the storage location in data bricks.
# As a workaround, i uploaded the geojson file to a public repository on GitHub.and downloded it her 
boroughList=[] 

for feature in data['features']: 
    polygon = shape(feature['geometry']) 
    boroughList.append({"Polygon_Size":polygon.area, "Borough_Name": feature['properties']['borough'],'Geometry':feature['geometry']})

sorted_boroughs = sorted(boroughList, key = lambda x: x['Polygon_Size'],reverse=True) 
                        
my_list_bc = sc.broadcast(sorted_boroughs) 

def GetBorough(longitude,latitude): 
    point = Point(longitude, latitude)
    boroughName="" 
    for feature in my_list_bc.value: 
        polygon = shape(feature['Geometry'])
        if polygon.contains(point):
            boroughName = feature['Borough_Name'] 
            break
    return boroughName 
GetBoroughName = udf(GetBorough,StringType())


-   The code creates a new DataFrame "df_enriched" based on an existing DataFrame "df".
-   Two new columns "Pickup_Location" and "Dropoff_Location" are added to "df_enriched" using the "withColumn" method.
-   The "GetBoroughName" function is called on two pairs of columns - ("pickup_longitude", "pickup_latitude") and ("dropoff_longitude", "dropoff_latitude").
-   The "GetBoroughName" function uses the Shapely library to match the latitude and longitude coordinates to a New York City borough, based on the geojson file loaded earlier.
-   The resulting borough name is then assigned to the corresponding "Pickup_Location" or "Dropoff_Location" column, respectively.
-   The backslash ("") at the end of the first line of code is used to indicate that the statement continues on the next line.

In [ ]:
print(type(df))



df_enriched = df.withColumn("Pickup_Location",
                            GetBoroughName ("pickup_longitude","pickup_latitude")).\
                                    withColumn ("Dropoff_Location",
                                                GetBoroughName ("dropoff_longitude", "dropoff_latitude"))


# display (df_enriched)


<class 'pyspark.sql.dataframe.DataFrame'>


-   The code updates the existing "df_enriched" DataFrame with additional columns and modifies existing ones.
-   A new column "Pickup_Location_N" is added using the "withColumn" method. This column is created by checking if the original "Pickup_Location" column is empty. If it is empty, then the "Dropoff_Location" column value is used. If it is not empty, then the original "Pickup_Location" value is used.
-   A new column "Dropoff_Location_N" is added in a similar manner, using the "withColumn" method. This column checks if the original "Dropoff_Location" column is empty, and if it is, then it uses the value from the "Pickup_Location_N" column instead. Otherwise, it uses the original "Dropoff_Location" value.
-   The original "Pickup_Location" and "Dropoff_Location" columns are dropped using the "drop" method on the "df_enriched" DataFrame.

In [ ]:
df_enriched = df_enriched.withColumn ("Pickup_Location_N",
                                      when(df_enriched.Pickup_Location==""
                                           ,df_enriched.Dropoff_Location)
                                      .otherwise(df_enriched.Pickup_Location))
df_enriched = df_enriched.withColumn("Dropoff_Location_N",
                                     when(df_enriched.Dropoff_Location=="",
                                          df_enriched. Pickup_Location_N)
                                     .otherwise(df_enriched.Dropoff_Location))
df_enriched = df_enriched.drop("Pickup_Location","Dropoff_Location")

# df_enriched= df_enriched.withColumn ("SameBorouh" ,when (df_enriched.Pickup_Location_N ==df_enriched.Dropoff_Location_N,"Y").otherwise("N"))
# display (df_enriched)



In [ ]:
df_enriched.printSchema() 


root
 |-- hack_license: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_time_in_secs: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)
 |-- pickup_timeStamp: timestamp (nullable = true)
 |-- dropoff_timeStamp: timestamp (nullable = true)
 |-- Pickup_Location_N: string (nullable = true)
 |-- Dropoff_Location_N: string (nullable = true)



In [ ]:
taxi_df = df_enriched.withColumn("fare_Duration", unix_timestamp("dropoff_timeStamp") - unix_timestamp("pickup_timeStamp"))
display (taxi_df)


hack_license,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,pickup_timeStamp,dropoff_timeStamp,Pickup_Location_N,Dropoff_Location_N,fare_Duration
BA96DE419E711691B9445D6A6307C170,4,382,1.00,-73.978165,40.757977,-73.989838,40.751171,2013-01-01T15:11:48.000+0000,2013-01-01T15:18:10.000+0000,Manhattan,Manhattan,382
9FD8F69F0804BDB5549F40E9DA1BE472,1,259,1.50,-74.006683,40.731781,-73.994499,40.75066,2013-01-06T00:18:35.000+0000,2013-01-06T00:22:54.000+0000,Manhattan,Manhattan,259
9FD8F69F0804BDB5549F40E9DA1BE472,1,282,1.10,-74.004707,40.73777,-74.009834,40.726002,2013-01-05T18:49:41.000+0000,2013-01-05T18:54:23.000+0000,Manhattan,Manhattan,282
51EE87E3205C985EF8431D850C786310,2,244,.70,-73.974602,40.759945,-73.984734,40.759388,2013-01-07T23:54:15.000+0000,2013-01-07T23:58:20.000+0000,Manhattan,Manhattan,245
51EE87E3205C985EF8431D850C786310,1,560,2.10,-73.97625,40.748528,-74.002586,40.747868,2013-01-07T23:25:03.000+0000,2013-01-07T23:34:24.000+0000,Manhattan,Manhattan,561
598CCE5B9C1918568DEE71F43CF26CD2,1,648,1.70,-73.966743,40.764252,-73.983322,40.743763,2013-01-07T15:27:48.000+0000,2013-01-07T15:38:37.000+0000,Manhattan,Manhattan,649
513189AD756FF14FE670D10B92FAF04C,1,418,.80,-73.995804,40.743977,-74.007416,40.744343,2013-01-08T11:01:15.000+0000,2013-01-08T11:08:14.000+0000,Manhattan,Manhattan,419
CCD4367B417ED6634D986F573A552A62,3,1898,10.70,-73.989937,40.756775,-73.86525,40.77063,2013-01-07T12:39:18.000+0000,2013-01-07T13:10:56.000+0000,Manhattan,Queens,1898
1DA2F6543A62B8ED934771661A9D2FA0,1,299,.80,-73.980072,40.743137,-73.982712,40.735336,2013-01-07T18:15:47.000+0000,2013-01-07T18:20:47.000+0000,Manhattan,Manhattan,300
CD2F522EEE1FF5F5A8D8B679E23576B3,2,957,2.50,-73.977936,40.786983,-73.952919,40.80637,2013-01-07T15:33:28.000+0000,2013-01-07T15:49:26.000+0000,Manhattan,Manhattan,958


In [ ]:
 taxi_df=taxi_df.withColumn("PickUp_Hour",hour("pickup_timeStamp"))  
taxi_df.write.option("header",True)\
    .partitionBy ("PickUp_Hour")\
    .mode("overwrite")\
    .parquet("/tmp/NYC_after")



In [ ]:
# Read the resulted data as parquet and create dataframe out of it
parDF1=spark.read.parquet("dbfs:/tmp/NYC_after")

#parDF1. createOrReplaceTempView("parquetTable")
parDF1.printSchema()



root
 |-- hack_license: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_time_in_secs: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)
 |-- pickup_timeStamp: timestamp (nullable = true)
 |-- dropoff_timeStamp: timestamp (nullable = true)
 |-- Pickup_Location_N: string (nullable = true)
 |-- Dropoff_Location_N: string (nullable = true)
 |-- fare_Duration: long (nullable = true)
 |-- PickUp_Hour: integer (nullable = true)



-   A new DataFrame `df_sorted` is created by sorting an existing DataFrame `parDF1` in descending order based on two columns: `hack_license` and `pickup_timestamp`.
-   The `filter` function is used twice on `df_sorted` to exclude any rows where `Pickup_Location_N` and `Dropoff_Location_N` are empty strings (`''`).
-   The `filter` function returns a new DataFrame that contains only the rows that meet the filter criteria.
-   `df_sorted` now contains only the rows where both pickup and dropoff locations are specified, and is sorted by `hack_license` and `pickup_timestamp` in descending order.

In [ ]:
# Drop Unknown Pickup or dropof location.
from pyspark.sql. functions import *

df_sorted =parDF1.orderBy(col("hack_License")
                          ,col("Pickup_timestaMp").desc())
df_sorted = df_sorted. filter (df_sorted.Pickup_Location_N !='').filter (df_sorted.Dropoff_Location_N != '')



-   A new window function `my_window` is defined with `hack_license` as the partitioning column and `Pickup_timestamp` as the ordering column.
-   `df_sorted` is further modified by adding a new column `Previous_Drop` that uses the window function `lag()` to get the previous row's `dropoff_timeStamp` value for each row in the partition.
-   Another new column `Idle_Minutes` is added to `df_sorted` using a combination of PySpark functions `when()`, `isnull()`, `round()`, and `otherwise()`. The column calculates the time interval between the current row's `pickup_timeStamp` and the previous row's `dropoff_timeStamp`, or `0` if there is no previous dropoff time. The resulting value is rounded to two decimal places and expressed in minutes.
-   A third new column `Idle_Hours` is added to `df_sorted` by dividing the `Idle_Minutes` column by `60`.

In [ ]:
from pyspark import SparkContext

from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.window import Window

my_window = Window. partitionBy('hack_License').orderBy("Pickup_timestamp")

df_sorted = df_sorted.withColumn("Previous_Drop", F.lag(df_sorted.dropoff_timeStamp).over(my_window))

df_sorted = df_sorted.withColumn("Idle_Minutes",
                                 F.round(
                                     F.when(F.isnull(df_sorted.Previous_Drop), 0)
                                         .otherwise(
                                             (df_sorted.pickup_timeStamp.cast("long") - df_sorted.Previous_Drop.cast("long")))/60,2))
df_sorted = df_sorted.withColumn("Idle_Hours", F.round(df_sorted.Idle_Minutes/60,2))
#todo round hours
display (df_sorted)


hack_license,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,pickup_timeStamp,dropoff_timeStamp,Pickup_Location_N,Dropoff_Location_N,fare_Duration,PickUp_Hour,Previous_Drop,Idle_Minutes,Idle_Hours
00AE05F56D451E89E6F83EBE384FF5E6,1,180,.60,-73.982658,40.723301,-73.974815,40.721645,2013-01-13T06:49:00.000+0000,2013-01-13T06:52:00.000+0000,Manhattan,Manhattan,180,6,null,0.0,0.0
00AE05F56D451E89E6F83EBE384FF5E6,1,420,1.38,-73.982269,40.728596,-74.001495,40.737251,2013-01-13T07:22:00.000+0000,2013-01-13T07:29:00.000+0000,Manhattan,Manhattan,420,7,2013-01-13T06:52:00.000+0000,30.0,0.5
00AE05F56D451E89E6F83EBE384FF5E6,1,660,2.76,-74.011322,40.707973,-73.989357,40.734238,2013-01-13T09:45:00.000+0000,2013-01-13T09:56:00.000+0000,Manhattan,Manhattan,660,9,2013-01-13T07:29:00.000+0000,136.0,2.27
00AE05F56D451E89E6F83EBE384FF5E6,1,540,3.03,-73.979881,40.729324,-73.971359,40.76366,2013-01-13T10:24:00.000+0000,2013-01-13T10:33:00.000+0000,Manhattan,Manhattan,540,10,2013-01-13T09:56:00.000+0000,28.0,0.47
00AE05F56D451E89E6F83EBE384FF5E6,1,600,2.03,-73.981026,40.74456,-73.985374,40.764668,2013-01-13T10:46:00.000+0000,2013-01-13T10:56:00.000+0000,Manhattan,Manhattan,600,10,2013-01-13T10:33:00.000+0000,13.0,0.22
00AE05F56D451E89E6F83EBE384FF5E6,1,780,3.90,-73.987411,40.739922,-73.955719,40.785133,2013-01-13T11:09:00.000+0000,2013-01-13T11:22:00.000+0000,Manhattan,Manhattan,780,11,2013-01-13T10:56:00.000+0000,13.0,0.22
00AE05F56D451E89E6F83EBE384FF5E6,1,720,2.27,-73.977821,40.729401,-73.995613,40.749222,2013-01-13T11:40:00.000+0000,2013-01-13T11:52:00.000+0000,Manhattan,Manhattan,720,11,2013-01-13T11:22:00.000+0000,18.0,0.3
00AE05F56D451E89E6F83EBE384FF5E6,1,840,2.74,-73.975578,40.733044,-74.001328,40.755474,2013-01-13T13:34:00.000+0000,2013-01-13T13:48:00.000+0000,Manhattan,Manhattan,840,13,2013-01-13T11:52:00.000+0000,102.0,1.7
00AE05F56D451E89E6F83EBE384FF5E6,1,420,1.80,-74.000031,40.746426,-73.983543,40.76543,2013-01-13T13:54:00.000+0000,2013-01-13T14:01:00.000+0000,Manhattan,Manhattan,420,13,2013-01-13T13:48:00.000+0000,6.0,0.1
00AE05F56D451E89E6F83EBE384FF5E6,1,420,.82,-73.974716,40.761852,-73.98465,40.757523,2013-01-13T14:09:00.000+0000,2013-01-13T14:16:00.000+0000,Manhattan,Manhattan,420,14,2013-01-13T14:01:00.000+0000,8.0,0.13


-   `df_filtered`: A new DataFrame created by filtering the previous `df_sorted` DataFrame based on the condition that the `Idle_Hours` column is less than or equal to 4.
-   `df_summary`: A new DataFrame created by grouping the `df_filtered` DataFrame by the `Dropoff_Location_N` column and calculating the total and average of the `Idle_Hours` column using the `agg()` function with `sum()` and `avg()` as aggregation functions, respectively.
-   `sum("Idle_Hours").alias("Total_Idle")`: calculates the total of the `Idle_Hours` column and renames it as `Total_Idle`.
-   `avg("Idle_Hours").alias("Average_Idle")`: calculates the average of the `Idle_Hours` column and renames it as `Average_Idle`.
-   `withColumn("Total_Idle", F.round(df_summary.Total_Idle,2))`: rounds the `Total_Idle` column values to 2 decimal points using the `round()` function from the `pyspark.sql.functions` module and updates the column in the DataFrame.
-   `withColumn("Average_Idle", F.round(df_summary.Average_Idle,2))`: rounds the `Average_Idle` column values to 2 decimal points using the `round()` function from the `pyspark.sql.functions` module and updates the column in the DataFrame.
-   `display(df_summary)`: displays the `df_summary` DataFrame in a tabular format.

In [ ]:
df_filtered = df_sorted.filter(df_sorted.Idle_Hours <=4)


df_summary = df_filtered.groupBy("Dropoff_Location_N").agg(sum("Idle_Hours").alias("Total_Idle"),\
            avg("Idle_Hours").alias("Average_Idle"))
df_summary = df_summary.withColumn("Total_Idle" 
                               ,F.round(df_summary.Total_Idle,2)).withColumn("Average_Idle"
                                ,F.round(df_summary.Average_Idle,2))

display (df_summary)


Dropoff_Location_N,Total_Idle,Average_Idle
Brooklyn,1124.52,0.32
Staten Island,1.17,0.09
Manhattan,21569.83,0.25
Queens,2147.24,0.39
Bronx,126.41,0.33


In [ ]:
# Compute the number of trips that started and ended within the same borough
same_borough_df = parDF1.filter(parDF1["Pickup_Location_N"] == parDF1["Dropoff_Location_N"]).count()
display(same_borough_df)


88566

In [ ]:
# Compute the number of trips that start in one borough and ended in another one
diff_borough_df = parDF1.filter(parDF1["Pickup_Location_N"] != parDF1["Dropoff_Location_N"]).count()
display (diff_borough_df)


11433